In [11]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

df = pd.read_json("../data/totto/linearized/train.jsonl", lines=True)

In [6]:
df.columns

Index(['table', 'table_webpage_url', 'table_page_title', 'table_section_title',
       'table_section_text', 'highlighted_cells', 'example_id',
       'sentence_annotations', 'full_table_str', 'subtable_str',
       'full_table_metadata_str', 'subtable_metadata_str'],
      dtype='object')

In [8]:
import json

tables = df["table"].apply(lambda x: pd.DataFrame(json.loads(x)))

In [9]:
titles = df["table_page_title"].values.tolist()

In [66]:
sentences = df["sentence_annotations"].apply(lambda x: x[0])

In [12]:
from kilt.knowledge_source import KnowledgeSource

ks = KnowledgeSource()

In [78]:
import regex as re

def number_matching(sentence, table):
    values = table.values.flatten().tolist()
    numbers = [x[0] for x in re.findall("(\d+(.\d+)?)", " ".join(values))]
    count = 0
    for number1 in re.findall("(\d+(.\d+)?)", sentence):
        for number2 in numbers:
            if number1[0] == number2:
                count += 1
                break
    return count >= 3
            
    
def row_matching(sentence, table):
    for idx, row in table.iterrows():
        count = 0

        for val1 in row.values:
            if val1 in sentence:
                count += 1
                break
        return count >= 3

In [82]:
from blingfire import *
examples = []

for title, table in list(zip(titles, tables))[:30]:
        page = ks.get_page_by_title(title)
        if page is None:
            examples.append(([], table))
            continue
            
        page = page["text"]
        t_sentences = []
        for passage in page:
            for sentence in text_to_sentences(passage).split("\n"):
                if number_matching(sentence, table) or row_matching(sentence, table):
                    t_sentences.append(sentence)
        
        examples.append((t_sentences, table))

In [83]:
for example, sentence, title in zip(examples, sentences, titles):
    print(example, sentence, title)
    print("\n")
    print("\n")
    print("\n")

([],      #                                   Run                 Title Chapters  \
0   59        June 5, 2000— February 2, 2001      Laços de Família      209   
1   60  February 5, 2001— September 28, 2001    Porto dos Milagres      203   
2   61        October 1, 2001— June 14, 2002               O Clone      221   
3   62      June 17, 2002— February 14, 2003             Esperança      209   
4   63   February 17, 2003— October 10, 2003  Mulheres Apaixonadas      203   
5   64       October 13, 2003— June 25, 2004           Celebridade      221   
6   65         June 28, 2004— March 11, 2005    Senhora do Destino      221   
7   66      March 14, 2005— November 4, 2005               América      203   
8   67        November 7, 2005— July 7, 2006             Belíssima      209   
9   68          July 10, 2006— March 2, 2007       Páginas da Vida      203   
10  69     March 5, 2007— September 28, 2007      Paraíso Tropical      179   
11  70         October 1, 2007— May 31, 2008   